In [3]:
import sqlite3
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.ticker import FuncFormatter
import seaborn as sns
from pathlib import Path
import matplotlib as mpl
from matplotlib.lines import Line2D
from scipy import stats
# set font size
mpl.rcParams.update({'font.size': 8})

# Set Helvetica font globally
mpl.rcParams['font.family'] = 'sans-serif'
mpl.rcParams['font.sans-serif'] = ['Helvetica']

# Optional: make sure fonts are updated
mpl.rcParams['pdf.fonttype'] = 42  # Avoids Type 3 fonts in PDF output
mpl.rcParams['ps.fonttype'] = 42

In [4]:
nerd_sqlite = '../../../Core_nerd_analysis/nerd.sqlite'

db_path = Path(nerd_sqlite).expanduser().resolve()
if not db_path.exists():
    raise FileNotFoundError(f"Database not found: {db_path}")

conn = sqlite3.connect(db_path)
conn.row_factory = sqlite3.Row

def read_sql(query: str, params: dict | None = None) -> pd.DataFrame:
    """Run a SQL query against the configured database and return a DataFrame."""
    return pd.read_sql_query(query, conn, params=params or {})

rg_ids = [123, 124, 129, 130]  # P4P6 rg_ids

query = """
SELECT *
FROM probe_tc_fits_view
WHERE fit_kind = 'round3_constrained'
  AND rg_id IN ({})
  AND rt_protocol = 'MRT'
ORDER BY rg_id
""".format(",".join(["?"] * len(rg_ids)))

df = read_sql(query, params=rg_ids)
conn.close()

# get max log_kobs for each nt_base and save in dict
max_logkobs = {}
for base in df['nt_base'].unique():
    df_base = df[df['nt_base'] == base]
    max_logkobs[base] = df_base['log_kobs'].max()
max_logkobs

def calc_dG(logkobs, logkadd):
    R = 1.9872036e-3  # kcal/(mol*K)
    T = 298.15  # K
    
    logKKp1 = logkobs - logkadd
    KKp1 = np.exp(logKKp1) # K / (K+1)
    K = KKp1 / (1 - KKp1)
    dG = -R * T * np.log(K)

    return dG


# calculate dG using base-specific max log_kobs
df['dG'] = df.apply(lambda row: calc_dG(row['log_kobs'], max_logkobs[row['nt_base']]), axis=1)


## Analysis of correlation

def keep_higher_r2(df):
    """
     keep_higher_r2 between duplicate RT runs of the same reactions
    """
    for site in df['nt_site'].unique():
        # drop row with lower r2
        df_site = df[df['nt_site'] == site]
        if len(df_site) != 2:
            continue
        if df_site.iloc[0]['r2'] > df_site.iloc[1]['r2']:
            df = df.drop(df_site.index[1])
        else:
            df = df.drop(df_site.index[0])
    return df

df_nomg = df[df['buffer_id'] == 2]
df_nomg = keep_higher_r2(df_nomg)
df_mg = df[df['buffer_id'] == 3]
df_mg = keep_higher_r2(df_mg)
df_mg

# merge df_nomg and df_mg on nt_site and keep the following: nt_site, nt_base, log_kobs, log_kobs_err, dG, r2
# add suffix to all but nt_site and nt_base
df_merged = pd.merge(df_nomg[['nt_site', 'nt_base', 'log_kobs', 'log_kobs_err', 'dG', 'r2']],
                     df_mg[['nt_site', 'log_kobs', 'log_kobs_err', 'dG', 'r2']],
                     on=['nt_site'],
                     suffixes=('_nomg', '_mg'))

# filter to A and C only
df_merged_AC = df_merged[df_merged['nt_base'].isin(['A', 'C'])]

# filter to R2 > 0.5
df_merged_AC = df_merged_AC[(df_merged_AC['r2_nomg'] > 0.5) & (df_merged_AC['r2_mg'] > 0.5)]

/var/folders/18/qp7ngpw963z9q34k6mzb5jc80000gn/T/ipykernel_22286/1683946227.py:41: RuntimeWarning: divide by zero encountered in scalar divide
  K = KKp1 / (1 - KKp1)


,nt_site,nt_base,log_kobs_nomg,log_kobs_err_nomg,dG_nomg,r2_nomg,log_kobs_mg,log_kobs_err_mg,dG_mg,r2_mg
0,104,A,-3.775280,0.039808,0.411026,0.981779,-3.795475,0.071300,0.428881,0.975766
1,105,A,-4.047125,0.045539,0.638676,0.976153,-4.165104,0.077981,0.730599,0.971924
4,109,C,-4.365995,0.035195,0.707925,0.985545,-4.499681,0.083820,0.809164,0.966432
5,113,A,-4.182189,0.028209,0.743639,0.990689,-4.636739,0.095737,1.071636,0.956610
6,114,A,-3.406555,0.027499,0.043210,0.991309,-3.458740,0.033007,0.101494,0.994756
...,...,...,...,...,...,...,...,...,...,...
116,261,A,-5.490267,0.085721,1.630607,0.919276,-5.448170,0.086865,1.604038,0.963795
117,262,A,-4.551276,0.056110,1.012275,0.963976,-4.814791,0.059075,1.192756,0.982990
118,263,C,-6.721343,0.215547,2.246925,0.642335,-6.297629,0.260076,1.988792,0.747605
120,265,A,-6.427110,0.171735,2.208222,0.738889,-6.461498,0.181311,2.229078,0.859019


In [6]:

# Load your dataframe (adapt as needed)
df = df_merged_AC.copy()

# Map nt_site -> dG_mg
dg_map = dict(zip(df["nt_site"], df["dG_mg"]))

input_pdb  = "1gid.pdb"              # original PDB
output_pdb = "1gid_with_dG.pdb"      # new PDB with dG in B-factor

with open(input_pdb) as fin, open(output_pdb, "w") as fout:
    for line in fin:
        if line.startswith(("ATOM", "HETATM")):
            chain_id = line[21]                 # column 22 in PDB (0-based index 21)
            resi_str = line[22:26]              # columns 23–26
            try:
                resi = int(resi_str)
            except ValueError:
                fout.write(line)
                continue

            # Here I assume your nt_site corresponds to chain A residue numbers
            # If you want other chains, adjust this condition
            if chain_id == "A" and resi in dg_map:
                b = dg_map[resi]                # dG_mg value
            else:
                # For residues without data (e.g. G/U), just give some neutral value
                # It won't matter if we later keep them gray in PyMOL.
                b = 0.0

            # Overwrite B-factor (columns 61–66, i.e. indices 60:66)
            new_line = line[:60] + f"{b:6.2f}" + line[66:]
            fout.write(new_line)
        else:
            fout.write(line)